In [1]:
import numpy as np
import tensorflow as tf

In [2]:
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2


In [3]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


In [4]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()


In [5]:
init = tf.global_variables_initializer() # prepare an init node
with tf.Session() as sess:
    init.run()  # initializes all the variables
    result = f.eval()
    

In [6]:
# interactive session...
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


In [7]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()


True

In [8]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
    print(x2.graph is tf.get_default_graph())
print(x2.graph is tf.get_default_graph())

tf.reset_default_graph()


True
False


In [9]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3
print("done")

with tf.Session() as sess:

    print("here")
    print(y.eval())
    print(z.eval())
    
print("done")

done
here
10
15
done


In [25]:
# page 238
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m,n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

x = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="x")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
xt = tf.transpose(x)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(xt,x)), xt), y)

with tf.Session()as sess:
    theta_value = theta.eval()
    


In [28]:
# page 239
# "Implementing Gradient Descent"
# this code is not on page 239 - it references "chapter 4" so I pulled
# it from the github for the book

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

print(scaled_housing_data_plus_bias.mean(axis=0))
print(scaled_housing_data_plus_bias.mean(axis=1))
print(scaled_housing_data_plus_bias.mean())
print(scaled_housing_data_plus_bias.shape)

[ 1.00000000e+00  6.60969987e-17  5.50808322e-18  6.60969987e-17
 -1.06030602e-16 -1.10161664e-17  3.44255201e-18 -1.07958431e-15
 -8.52651283e-15]
[ 0.38915536  0.36424355  0.5116157  ... -0.06612179 -0.06360587
  0.01359031]
0.11111111111111005
(20640, 9)


In [34]:
# page 239, 240, 241
# page 240 replaces one line the 'gradient')
# page 241 replaces two lines near the 'gradient

# NOTE this code does not run successfully - prints NAN for
# each eposh and then fails after a few epochs. I suspect it is
# lines 11,12 where the book calls for "scaled_housing_data_plus_bias"
# I cannot find this value in the examples from the book so I used
# a similar (non-scaled?) variable from earlier examples.

#reset_graph()

n_epochs = 1000
learning_rate = 0.01

x = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="x")
y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(x, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')

##gradients = 2/n + tf.matmul(tf.transpose(x), error)
#gradients = tf.gradients(mse, [theta])[0]
#training_op = tf.assign(theta, theta - learning_rate * gradients)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

print(best_theta)


Epoch 0 MSE = 3.6645076
Epoch 100 MSE = 0.82145524
Epoch 200 MSE = 0.6801511
Epoch 300 MSE = 0.63510466
Epoch 400 MSE = 0.60543585
Epoch 500 MSE = 0.5840003
Epoch 600 MSE = 0.5683552
Epoch 700 MSE = 0.5569065
Epoch 800 MSE = 0.54851115
Epoch 900 MSE = 0.54234135
[[ 2.0685525 ]
 [ 0.8381321 ]
 [ 0.15952322]
 [-0.2083361 ]
 [ 0.22617947]
 [ 0.01021773]
 [-0.04295653]
 [-0.5946695 ]
 [-0.5627706 ]]


In [36]:
# pages 241, 242 - Placeholder Nodes (Feeding Data to the Training Algorithm)
# using mini-batch Gradient Descent as an example...

n_epochs = 1000
learning_rate = 0.01

#reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

print(best_theta)

[[ 2.070016  ]
 [ 0.8204561 ]
 [ 0.1173173 ]
 [-0.22739051]
 [ 0.3113402 ]
 [ 0.00353193]
 [-0.01126994]
 [-0.91643935]
 [-0.8795008 ]]


In [38]:
# Saving and Restoring Models (page 243)
#reset_graph()

n_epochs = 1000                                                                       # not shown in the book
learning_rate = 0.01                                                                  # not shown

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")            # not shown
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")            # not shown
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")                                      # not shown
error = y_pred - y                                                                    # not shown
mse = tf.reduce_mean(tf.square(error), name="mse")                                    # not shown
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)            # not shown
training_op = optimizer.minimize(mse)                                                 # not shown

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())                                # not shown
            save_path = saver.save(sess, "/tmp/my_model.ckpt")
        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt")
    
print(best_theta)



Epoch 0 MSE = 2.7544272
Epoch 100 MSE = 0.63222194
Epoch 200 MSE = 0.5727796
Epoch 300 MSE = 0.5585005
Epoch 400 MSE = 0.54906934
Epoch 500 MSE = 0.5422877
Epoch 600 MSE = 0.5373788
Epoch 700 MSE = 0.5338219
Epoch 800 MSE = 0.5312427
Epoch 900 MSE = 0.52937055
[[ 2.06855249e+00]
 [ 7.74078071e-01]
 [ 1.31192386e-01]
 [-1.17845066e-01]
 [ 1.64778143e-01]
 [ 7.44078017e-04]
 [-3.91945094e-02]
 [-8.61356676e-01]
 [-8.23479772e-01]]
INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


True

In [40]:
with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")
    best_theta_restored = theta.eval() # not shown in the book

np.allclose(best_theta, best_theta_restored)


INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


True

In [41]:
# comment from the code from the book's git:
#If you want to have a saver that loads and restores theta with a different name, such as "weights":
saver = tf.train.Saver({"weights": theta})

# By default the saver also saves the graph structure itself in a second file
# with the extension .meta. You can use the function tf.train.import_meta_graph() to
# restore the graph structure. This function loads the graph into the default graph and
# returns a Saver that can then be used to restore the graph state (i.e., the variable values):

#reset_graph()
# notice that we start with an empty graph.

saver = tf.train.import_meta_graph("/tmp/my_model_final.ckpt.meta")  # this loads the graph structure
theta = tf.get_default_graph().get_tensor_by_name("theta:0") # not shown in the book

with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")  # this restores the graph's state
    best_theta_restored = theta.eval() # not shown in the book

print(np.allclose(best_theta, best_theta_restored))

# This means that you can import a pretrained model without having to have the
# corresponding Python code to build the graph. This is very handy when you keep
# tweaking and saving your model: you can load a previously saved model without
# having to search for the version of the code that built it.

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt
False


In [39]:
# Visualizing the Graph and Training Curves using TensorBoard
# page 244

from tensorflow_graph_in_jupyter import show_graph
show_graph(tf.get_default_graph())


ImportError: No module named 'tensorflow_graph_in_jupyter'